# Metrics for measuring a split.
* We'll develop functions to compute the following metrics to measure the
quality of a separation
1. Gini impurity index: A lower Gini impurity index indicates a purer dataset.
2. Information gain (entropy): Measures the improvement of purity after
splitting. In other words, the reduction of uncertainty due to a split.
3. Entropy: Is a probabilistic measure of uncertainty. Lower entropy implies
a purer dataset with less ambiguity.


## Gini impurity index

In [105]:
# Programming a function to calculate the gini impurity index
import numpy as np

def gini_impurity(labels):
    """
    Calculate the Gini impurity of a list of labels.

    :param labels: List of labels in a dataset.
    :return: Gini impurity index.
    """
    # When the set is empty, it is also pure
    if not labels:
        return 0
    # Count the occurrences of each label
    counts = np.unique(labels, return_counts=True)[1]
    # Calculate the fractions of each label's occurrences
    fractions = counts / float(len(labels))
    # Compute the Gini impurity index
    return 1 - np.sum(fractions ** 2)

In [106]:
# Testing out some examples:
print(f"{gini_impurity([1, 1, 0, 1, 1, 0, 1]):.4f}")

0.4082


In [107]:
print(f'{gini_impurity([1, 1, 0, 1, 0, 0]):.4f}')

0.5000


## Information gain and Entropy


In [108]:
# Calculating the entropy of a given set
def entropy(labels):
    """
    Calculate the entropy of a list of labels.

    :param labels: List of labels in a dataset.
    :return: Entropy of the dataset.
    """
    # When the set is empty, it has no entropy
    if not labels:
        return 0
    # Count the occurrences of each label
    counts = np.unique(labels, return_counts=True)[1]
    # Calculate the fractions of each label's occurrences
    fractions = counts / float(len(labels))
    # Compute the entropy using the fractions and log base 2
    return - np.sum(fractions * np.log2(fractions))

In [109]:
# Checking some examples:
print(f"{entropy([1, 1, 0, 1, 1, 0, 1]):.4f}")

0.8631


In [110]:
print(f'{entropy([1, 1, 0, 1, 0]):.4f}')

0.9710


In [111]:
print(f'{entropy([1, 1, 1, 1]):.4f}')

-0.0000


In [112]:
# Define a dictionary to map criterion names to their respective functions
criterion_function = {'gini': gini_impurity, 'entropy': entropy}

def weighted_impurity(groups, criterion='gini'):
    """
    Calculate the weighted impurity of children after a split.

    :param groups: List of children, and a child consists of a list of labels.
    :param criterion: Metric to measure the quality of a split. 'gini' for
                      Gini Impurity and 'entropy' for Information Gain.
    :return: float: Weighted impurity.
    """
    # Calculate the total number of samples in all groups
    total = sum(len(group) for group in groups)
    # Initialize the weighted sum of impurities
    weighted_sum = 0.0

    # Iterate through each group
    for group in groups:
        # Calculate the weight of the group (proportion of total samples)
        weight = len(group) / float(total)
        # Calculate the impurity of the group using the selected criterion function
        impurity = criterion_function[criterion](group)
        # Add the weighted impurity to the running sum
        weighted_sum += weight * impurity

    # Return the final weighted impurity value
    return weighted_sum

To understand this, we'll calculate the entropy for a toy example and we'll
look for the best possible split:
![Image Description](04_splits.png)



In [113]:
# Testing with the toy example:
children_1 = [[1, 0, 1], [0, 1]] # Split by gender
children_2 = [[1, 1], [0, 0, 1]] # Split by interest in tech

print(f"Entropy of #1 split: {weighted_impurity(children_1, 'entropy'):.4f}")
print(f"Entropy of #2 split: {weighted_impurity(children_2, 'entropy'):.4f}")

Entropy of #1 split: 0.9510
Entropy of #2 split: 0.5510


This means that the best split is the second one (based on interest in tech.

## Implementing the CART algorithm by scratch
* We'll build a tree based on the best split (the one that either minimizes
the Gini Impurity Index or the one that minimizes Entropy)
* The plan is to find the best split for the following toy dataset
![Image Description](04_toy_dataset_uinterset_uoccupation.png)

In [114]:
# We need to rewrite our previous functions in order to handle numpy arrays
# as inputs:
# Programming a function to calculate the gini impurity index
import numpy as np

def gini_impurity_np(labels):
    """
    Calculate the Gini impurity of a list of labels.

    :param labels: List of labels in a dataset.
    :return: Gini impurity index.
    """
    # When the set is empty, it is also pure
    if labels.size == 0:
        return 0
    # Count the occurrences of each label
    counts = np.unique(labels, return_counts=True)[1]
    # Calculate the fractions of each label's occurrences
    fractions = counts / float(len(labels))
    # Compute the Gini impurity index
    return 1 - np.sum(fractions ** 2)

# Calculating the entropy of a given set
def entropy_np(labels):
    """
    Calculate the entropy of a list of labels.

    :param labels: List of labels in a dataset.
    :return: Entropy of the dataset.
    """
    # When the set is empty, it has no entropy
    if labels.size == 0:
        return 0
    # Count the occurrences of each label
    counts = np.unique(labels, return_counts=True)[1]
    # Calculate the fractions of each label's occurrences
    fractions = counts / float(len(labels))
    # Compute the entropy using the fractions and log base 2
    return - np.sum(fractions * np.log2(fractions))

# Define a dictionary to map criterion names to their respective functions
criterion_function_np = {'gini': gini_impurity, 'entropy': entropy}

def weighted_impurity(groups, criterion='gini'):
    """
    Calculate the weighted impurity of children after a split.

    :param groups: List of children, and a child consists of a list of labels.
    :param criterion: Metric to measure the quality of a split. 'gini' for
                      Gini Impurity and 'entropy' for Information Gain.
    :return: float: Weighted impurity.
    """
    # Calculate the total number of samples in all groups
    total = sum(len(group) for group in groups)
    # Initialize the weighted sum of impurities
    weighted_sum = 0.0

    # Iterate through each group
    for group in groups:
        weighted_sum += len(group) / float(total) * \
                        criterion_function_np[criterion](group)

    # Return the final weighted impurity value
    return weighted_sum

In [115]:
# The function should take three arguments: the left group's labels, the right group's labels, and the criterion (either 'gini' or 'entropy'). Here's the correct definition for the weighted_impurity function:

def weighted_impurity(left_labels, right_labels, criterion):
    total_samples = len(left_labels) + len(right_labels)
    if criterion == 'gini':
        return (len(left_labels) / total_samples) * gini_impurity_np(left_labels) + \
               (len(right_labels) / total_samples) * gini_impurity_np(right_labels)
    elif criterion == 'entropy':
        return (len(left_labels) / total_samples) * entropy_np(left_labels) + \
               (len(right_labels) / total_samples) * entropy_np(right_labels)
    else:
        raise ValueError("Invalid criterion value. Choose either 'gini' or 'entropy'.")

In [116]:
# Defining a utility function to split a node into left and right children,
# based on a feature and a value (target)
def split_node(X, y, index, value):
    """
    Splits the dataset X, y based on a feature and a value
    @param X: numpy.ndarray, a feature of the dataset
    @param y: numpy.ndarray, a target of the dataset
    @param index: int, index of the feature used for splitting [column]
    @param value: value of the feature used for splitting [unique value]
    @return: list, list, left and right child, a child is in the format of
    [X, y]
    """
    # All of the rows from the feature column
    x_index = X[:, index]
    # If the feature is numerical
    if X[0, index].dtype.kind in ['i', 'f']:
        mask = x_index >= value
    # If the feature is categorical
    else:
        mask = x_index == value
    # The left child contains rows of X and y for which the mask array has False values (using the bitwise NOT operator ~ to invert the mask).
    left = [X[~mask, :], y[~mask]]
    #  The right child contains rows of X and y for which the mask array has True values.
    right = [X[mask, :], y[mask]]
    return left, right


In [117]:
# We now implement the greedy search function, which tries out all possible
# splits and returns the best one given a selection criterion, along with the
# resulting children:
def get_best_split(X, y, criterion):
    # Initialize variables to keep track of the best split found so far
    best_index, best_value, best_score, children = None, None, 1, None

    # Iterate through all features (columns) in the dataset X
    for index in range(len(X[0])):
        # Iterate through all unique values in the current feature column
        for value in np.sort(np.unique(X[:, index])):
            # Split the dataset based on the current feature and value
            groups = split_node(X, y, index, value)
            # Calculate the weighted impurity of the resulting children groups
            impurity = weighted_impurity(groups[0][1], groups[1][1], criterion)
            # Check if the current split is better than the best split found so far
            if impurity < best_score:
                # Update the best split information (index, value, score, and children)
                best_index, best_value, best_score, children = index, value, \
                                                               impurity, groups

    # Return the best split information as a dictionary
    return {'index': best_index, 'value': best_value, 'children': children}


In [118]:
# The most frequent class (label) among the training data points that have
# reached that leaf node. In other words, it is the most common class among the samples that belong to the leaf node.
def get_leaf(labels):
    # Obtain the leaf as the majority of the labels
    return np.bincount(labels).argmax()

# The following recursive function links everything together.
* It assigns a leaf node if one of two child nodes is empty
* It assigns a leaf node if the current branch depth exceeds the maximum
depth allowed.
* It assigns a leaf node if the node does not contain sufficient samples
required for a further split.
* Otherwise, it proceeds with a further split with the optimal splitting point.
* This function is responsible for recursively constructing the decision tree by splitting the nodes based on the best feature and value, while also taking into consideration the stopping criteria (max_depth and min_size). The tree is grown until the stopping criteria are met or the nodes become pure, meaning they contain samples of only one

In [119]:
def split(node, max_depth, min_size, depth, criterion):
    """
    Split children of a node to construct new nodes or assign them terminals.
    @param node: dict, with children info
    @param max_depth: int, maximal depth of the tree
    @param min_size: int, minimal samples required to further split a child
    @param depth: int, current depth of the node
    @param criterion: gini or entropy
    @return: null
    """
    # Unpack the children (left and right) from the input node dictionary
    left, right = node['children']

    # If there are no samples in the left child, assign the majority label of the right child to the node and return
    if left[1].size == 0:
        node['right'] = get_leaf(right[1])
        return
    # If there are no samples in the right child, assign the majority label of the left child to the node and return
    if right[1].size == 0:
        node['left'] = get_leaf(left[1])
        return

    # Check if the current depth exceeds the maximal depth allowed for the tree
    if depth >= max_depth:
        # If it exceeds the maximal depth, assign majority labels to both children and return
        node['left'], node['right'] = get_leaf(left[1]), get_leaf(right[1])
        return

    # Check if the left child has enough samples to be further split
    if left[1].size <= min_size:
        # If not, assign the majority label to the left child
        node['left'] = get_leaf(left[1])
    else:
        # If it has enough samples, further split the left child
        result = get_best_split(left[0], left[1], criterion)
        result_left, result_right = result['children']
        # Check if there are no samples in the left or right child of the split
        if result_left[1].size == 0:
            node['left'] = get_leaf(result_right[1])
        elif result_right[1].size == 0:
            node['left'] = get_leaf(result_left[1])
        else:
            # If both children have samples, store the split information in the node and continue splitting
            node['left'] = result
            split(node['left'], max_depth, min_size, depth + 1, criterion)

    # Check if the right child has enough samples to be further split
    if right[1].size <= min_size:
        # If not, assign the majority label to the right child
        node['right'] = get_leaf(right[1])
    else:
        # If it has enough samples, further split the right child
        result = get_best_split(right[0], right[1], criterion)
        result_left, result_right = result['children']
        # Check if there are no samples in the left or right child of the split
        if result_left[1].size == 0:
            node['right'] = get_leaf(result_right[1])
        elif result_right[1].size == 0:
            node['right'] = get_leaf(result_left[1])
        else:
            # If both children have samples, store the split information in the node and continue splitting
            node['right'] = result
            split(node['right'], max_depth, min_size, depth + 1, criterion)

The entry point of the tree's construction is as follows

In [120]:
def train_tree(X_train, y_train, max_depth, min_size, criterion = 'gini'):
    X = np.array(X_train)
    y = np.array(y_train)
    root = get_best_split(X, y, criterion)
    split(root, max_depth, min_size, 1, criterion)
    return root



In [121]:
# Simulating the toy dataset
X_train = [['tech', 'professional'],
           ['fashion', 'student'],
           ['fashion', 'professional'],
           ['sports', 'student'],
           ['tech', 'student'],
           ['tech', 'retired'],
           ['sports', 'professional']]
y_train = [1, 0, 0, 0, 1, 0, 1]
tree = train_tree(X_train, y_train, 2, 2)
tree

{'index': 0,
 'value': 'fashion',
 'children': ([array([['tech', 'professional'],
          ['sports', 'student'],
          ['tech', 'student'],
          ['tech', 'retired'],
          ['sports', 'professional']], dtype='<U12'),
   array([1, 0, 1, 0, 1])],
  [array([['fashion', 'student'],
          ['fashion', 'professional']], dtype='<U12'),
   array([0, 0])]),
 'left': {'index': 1,
  'value': 'professional',
  'children': ([array([['sports', 'student'],
           ['tech', 'student'],
           ['tech', 'retired']], dtype='<U12'),
    array([0, 1, 0])],
   [array([['tech', 'professional'],
           ['sports', 'professional']], dtype='<U12'),
    array([1, 1])]),
  'left': 0,
  'right': 1},
 'right': 0}

In [123]:
# Auxiliar function to visualize the tree that we built using the CART
# algorithm.
CONDITION = {'numerical': {'yes': '>=', 'no': '<'},
              'categorical': {'yes': 'is', 'no': 'is not'}}
def visualize_tree(node, depth=0):
     if isinstance(node, dict):
         if node['value'].dtype.kind in ['i', 'f']:
             condition = CONDITION['numerical']
         else:
             condition = CONDITION['categorical']
         print('{}|- X{} {} {}'.format(depth * '  ',
             node['index'] + 1, condition['no'], node['value']))
         if 'left' in node:
             visualize_tree(node['left'], depth + 1)
         print('{}|- X{} {} {}'.format(depth * '  ',
             node['index'] + 1, condition['yes'], node['value']))
         if 'right' in node:
             visualize_tree(node['right'], depth + 1)
     else:
         print(f"{depth * '  '}[{node}]")
visualize_tree(tree)

|- X1 is not fashion
  |- X2 is not professional
    [0]
  |- X2 is professional
    [1]
|- X1 is fashion
  [0]


The previous result is a representation of the best split possible. It's
based on the one we built by hand using the gini impurity index.
![Image Description](04_toy_dataset_gini_split.png)


In [124]:
# Testing a numerical example
X_train_n = [[6, 7],
             [2, 4],
             [7, 2],
             [3, 6],
             [4, 7],
             [5, 2],
             [1, 6],
             [2, 0],
             [6, 3],
             [4, 1]]
y_train_n = [0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
tree = train_tree(X_train_n, y_train_n, 2, 2)
visualize_tree(tree)

|- X2 < 4
  |- X1 < 7
    [1]
  |- X1 >= 7
    [0]
|- X2 >= 4
  |- X1 < 2
    [1]
  |- X1 >= 2
    [0]
